In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
def map_label(lbl):
    mapping = {
        'null': 0,
        'jogging': 1,
        'jogging (rotating arms)': 2,
        'jogging (skipping)': 3,
        'jogging (sidesteps)': 4,
        'jogging (butt-kicks)': 5,
        'stretching (triceps)': 6,
        'stretching (lunging)': 7,
        'stretching (shoulders)': 8,
        'stretching (hamstrings)': 9,
        'stretching (lumbar rotation)': 10,
        'push-ups': 11,
        'push-ups (complex)': 12,
        'sit-ups': 13,
        'sit-ups (complex)': 14,
        'burpees': 15,
        'lunges': 16,
        'lunges (complex)': 17,
        'bench-dips': 18
    }
    return mapping.get(lbl, np.nan)
label_map = {
    'null': 0,'jogging': 1,'jogging (rotating arms)': 2,'jogging (skipping)': 3,'jogging (sidesteps)': 4,'jogging (butt-kicks)': 5,
    'stretching (triceps)': 6,'stretching (lunging)': 7,'stretching (shoulders)': 8,'stretching (hamstrings)': 9,'stretching (lumbar rotation)': 10,
    'push-ups': 11,'push-ups (complex)': 12,'sit-ups': 13,'sit-ups (complex)': 14,'burpees': 15,'lunges': 16,'lunges (complex)': 17,'bench-dips': 18
}

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [4]:
data_dir = Path('data')
train_dir = data_dir / 'train'
meta_file = data_dir / 'meta_data.txt'
test_file = data_dir/'test.csv'

In [5]:
sbj_files = sorted(train_dir.glob('sbj_*.csv'))
dfs = []
for f in sbj_files:
    df = pd.read_csv(f,low_memory=False)
    df['subject'] = df['sbj_id'].astype(str)
    dfs.append(df)

raw_df = pd.concat(dfs, ignore_index=True)


In [6]:
raw_df['label_code'] = raw_df['label'].apply(map_label)
raw_df = raw_df.dropna(subset=['label_code']).reset_index(drop=True)
raw_df['label_code'] = raw_df['label_code'].astype(int)

sensor_cols = [c for c in raw_df.columns if c not in ['sbj_id', 'subject', 'label', 'label_code']]
raw_df = raw_df.dropna(subset=sensor_cols).reset_index(drop=True)
scaler = StandardScaler()
raw_df[sensor_cols] = scaler.fit_transform(raw_df[sensor_cols])

In [7]:
def create_sequences(df, sensor_cols, target_col, window, step):
    X, y = [], []
    data = df[sensor_cols].values
    labels = df[target_col].values
    for start in range(0, len(df) - window + 1, step):
        end = start + window
        seq = data[start:end]
        lab = np.bincount(labels[start:end]).argmax()
        X.append(seq)
        y.append(lab)
    return np.array(X), np.array(y)

In [8]:
class SensorDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

In [9]:
def jitter(x, sigma=0.8):
    return x + np.random.normal(loc=0., scale=sigma, size=x.shape)


def scaling(x, sigma=1.1):
    """
    Applies scaling to a 2D tensor of shape (sequence_length, features).
    """
    # The factor should have the same shape as the input to allow for element-wise multiplication.
    factor = np.random.normal(loc=1., scale=sigma, size=x.shape)
    return x * factor

class ContrastiveDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]

        # Apply two different augmentations
        x1 = torch.tensor(jitter(x.numpy()), dtype=torch.float32)
        x2 = torch.tensor(scaling(x.numpy()), dtype=torch.float32)

        return x1, x2, y

In [10]:
class NTXentLoss(torch.nn.Module):

    def __init__(self, device, batch_size, temperature=0.1, use_cosine_similarity=True):
        super(NTXentLoss, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.device = device
        self.softmax = torch.nn.Softmax(dim=-1)
        self.mask_samples_from_same_repr = self._get_correlated_mask().type(torch.bool)
        self.similarity_function = self._get_similarity_function(use_cosine_similarity)
        self.criterion = torch.nn.CrossEntropyLoss(reduction="sum")

    def _get_similarity_function(self, use_cosine_similarity):
        if use_cosine_similarity:
            self._cosine_similarity = torch.nn.CosineSimilarity(dim=-1)
            return self._cosine_simililarity
        else:
            return self._dot_simililarity

    def _get_correlated_mask(self):
        diag = np.eye(2 * self.batch_size)
        l1 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=-self.batch_size)
        l2 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=self.batch_size)
        mask = torch.from_numpy((diag + l1 + l2))
        mask = (1 - mask).type(torch.bool)
        return mask.to(self.device)

    @staticmethod
    def _dot_simililarity(x, y):
        v = torch.tensordot(x.unsqueeze(1), y.T.unsqueeze(0), dims=2)
        return v

    def _cosine_simililarity(self, x, y):
        v = self._cosine_similarity(x.unsqueeze(1), y.unsqueeze(0))
        return v

    def forward(self, zis, zjs):
        representations = torch.cat([zjs, zis], dim=0)
        similarity_matrix = self.similarity_function(representations, representations)
        l_pos = torch.diag(similarity_matrix, self.batch_size)
        r_pos = torch.diag(similarity_matrix, -self.batch_size)
        positives = torch.cat([l_pos, r_pos]).view(2 * self.batch_size, 1)
        negatives = similarity_matrix[self.mask_samples_from_same_repr].view(2 * self.batch_size, -1)
        logits = torch.cat((positives, negatives), dim=1)
        logits /= self.temperature
        labels = torch.zeros(2 * self.batch_size).to(self.device).long()
        loss = self.criterion(logits, labels)
        return loss / (2 * self.batch_size)

In [11]:
class DeepConvLSTM_contrastive(nn.Module):
    def __init__(self, num_channels, embedding_dim=128):
        super().__init__()
        # Backbone
        self.conv1 = nn.Conv1d(num_channels, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.conv3 = nn.Conv1d(128, 128, kernel_size=5, padding=2)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.lstm = nn.LSTM(128, 128, num_layers=2, batch_first=True)

        # Projection Head
        self.projection = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )

    def forward(self, x):
        # Backbone forward pass
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x)); x = self.pool(x)
        x = self.relu(self.conv2(x)); x = self.pool(x)
        x = self.relu(self.conv3(x)); x = self.pool(x)
        x = x.permute(0, 2, 1)
        out, _ = self.lstm(x)
        features = out[:, -1, :]

        # Projection head forward pass
        projection = self.projection(features)

        return features, projection

In [12]:

class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, num_tokens=16, num_heads=4, num_layers=2, dropout=0.2):
        super().__init__()

        # Ensure the input can be reshaped into tokens cleanly
        assert input_dim % num_tokens == 0, "input_dim must be divisible by num_tokens"
        self.token_dim = input_dim // num_tokens
        self.num_tokens = num_tokens
        self.input_dim = input_dim

        # A standard PyTorch Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.token_dim,
            nhead=num_heads,
            dropout=dropout,
            batch_first=True  # Important: ensures input format is (batch, seq, feature)
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # A linear layer to make the final classification
        self.classifier_head = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        # x starts with shape: (batch_size, input_dim), e.g., (64, 128)

        # 1. Reshape the vector into a sequence of tokens
        # Shape becomes: (batch_size, num_tokens, token_dim), e.g., (64, 16, 8)
        x = x.view(-1, self.num_tokens, self.token_dim)

        # 2. Pass the sequence through the Transformer Encoder
        transformer_output = self.transformer_encoder(x)

        # 3. Flatten the output sequence back into a single vector
        # Shape becomes: (batch_size, input_dim), e.g., (64, 128)
        x_flat = transformer_output.reshape(-1, self.input_dim)

        # 4. Make the final prediction
        return self.classifier_head(x_flat)


In [13]:

raw_df_filtered = raw_df[raw_df['label_code'] != 0].copy()

# Create a new mapping for the remaining labels (1-18 -> 0-17)
# This is crucial for the model's loss function
original_labels = sorted(raw_df_filtered['label_code'].unique())
label_remapping = {orig_label: new_label for new_label, orig_label in enumerate(original_labels)}
raw_df_filtered['remapped_label'] = raw_df_filtered['label_code'].map(label_remapping)
right_arm_df = raw_df_filtered[["right_arm_acc_x", "right_arm_acc_y", "right_arm_acc_z", "subject", "remapped_label"]]
left_arm_df = raw_df_filtered[["left_arm_acc_x", "left_arm_acc_y", "left_arm_acc_z", "subject", "remapped_label"]]
right_leg_df = raw_df_filtered[["right_leg_acc_x", "right_leg_acc_y", "right_leg_acc_z", "subject", "remapped_label"]]
left_leg_df = raw_df_filtered[["left_leg_acc_x", "left_leg_acc_y", "left_leg_acc_z", "subject", "remapped_label"]]


In [14]:
from torchview import draw_graph


In [18]:
WINDOW_SIZE = 50
STEP_SIZE = 25
for i, df in enumerate([right_arm_df, left_arm_df, left_leg_df, right_leg_df]):
    all_X, all_y = [], []
    for subj in df['subject'].unique():
        df_sub = df[df['subject'] == subj].reset_index(drop=True)
        X_sub, y_sub = create_sequences(df_sub, [c for c in df.columns if c not in ['sbj_id', 'subject', 'label', 'label_code', 'remapped_label']], 'remapped_label', WINDOW_SIZE, STEP_SIZE)
        all_X.append(X_sub)
        all_y.append(y_sub)
    X = np.vstack(all_X)
    y = np.hstack(all_y)
    print(f"X shape: {X.shape}, y shape: {y.shape}")

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    torch.manual_seed(42)
    batch_size = 64
    train_loader = DataLoader(SensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(SensorDataset(X_val, y_val), batch_size=batch_size)

    d = 3
    num_classes = 18
    contrastive_loader = DataLoader(ContrastiveDataset(X_train, y_train), batch_size=batch_size, shuffle=True, drop_last=True)

    contrastive_model = DeepConvLSTM_contrastive(d).to(device)
    contrastive_optimizer = torch.optim.Adam(contrastive_model.parameters(), lr=1e-3)
    contrastive_criterion = NTXentLoss(device=device, batch_size=batch_size, temperature=0.5)
    model_graph = draw_graph(contrastive_model, graph_name="ContrastModelTransformer", input_size=(batch_size, 50, 3), device='cpu', roll=True, save_graph=True)

    CONTRASTIVE_EPOCHS = 10
    for epoch in range(1, CONTRASTIVE_EPOCHS + 1):
        contrastive_model.train()
        total_loss = 0
        for x1, x2, _ in tqdm(contrastive_loader, desc=f"Epoch {epoch}"):
            x1, x2 = x1.to(device), x2.to(device)

            contrastive_optimizer.zero_grad()

            _, proj1 = contrastive_model(x1)
            _, proj2 = contrastive_model(x2)

            loss = contrastive_criterion(proj1, proj2)

            loss.backward()
            contrastive_optimizer.step()

            total_loss += loss.item() * x1.size(0)

        avg_loss = total_loss / len(contrastive_loader.dataset)
        print(f"Epoch {epoch}/{CONTRASTIVE_EPOCHS} - Contrastive Loss: {avg_loss:.4f}")

    for param in contrastive_model.parameters():
        param.requires_grad = False

    classifier = TransformerClassifier(
    input_dim=128,
    num_classes=num_classes
    ).to(device)

    classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4) # Transformers often prefer a smaller learning rate
    classifier_criterion = nn.CrossEntropyLoss()
    model_graph = draw_graph(contrastive_model, graph_name="ClassifierModelTransformer", input_size=(batch_size, 1, 128), device='cpu', roll=True, save_graph=True)

    FINETUNE_EPOCHS = 20

    train_losses, val_losses = [], []
    for epoch in range(1, FINETUNE_EPOCHS + 1):
        contrastive_model.eval() # Backbone is in eval mode
        classifier.train()       # Classifier is in train mode
        train_loss = 0
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            classifier_optimizer.zero_grad()

            # Get features from the frozen contrastive model
            # The permute operation is correctly handled inside its forward pass
            with torch.no_grad():
                features, _ = contrastive_model(Xb)

            # Train the classifier
            preds = classifier(features)
            loss = classifier_criterion(preds, yb)
            loss.backward()
            classifier_optimizer.step()
            train_loss += loss.item() * Xb.size(0)
        train_losses.append(train_loss / len(train_loader.dataset))

        contrastive_model.eval()
        classifier.eval()
        val_loss, correct = 0, 0
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                features, _ = contrastive_model(Xb)
                preds = classifier(features)
                val_loss += classifier_criterion(preds, yb).item() * Xb.size(0)
                correct += (preds.argmax(1) == yb).sum().item()
        val_losses.append(val_loss / len(val_loader.dataset))
        print(f"Epoch {epoch}/{FINETUNE_EPOCHS} - Train: {train_losses[-1]:.4f}, Val: {val_losses[-1]:.4f}, Acc: {correct / len(val_loader.dataset):.4f}")
    torch.save(contrastive_model, f"models/DeepConvContrast/2_{i}_contrast.pt")
    torch.save(classifier, f"models/DeepConvContrast/2_{i}_classifier.pt")




X shape: (82157, 50, 3), y shape: (82157,)


Epoch 1: 100%|██████████| 1026/1026 [03:15<00:00,  5.25it/s]


Epoch 1/10 - Contrastive Loss: 3.2398


Epoch 2: 100%|██████████| 1026/1026 [04:00<00:00,  4.26it/s]


Epoch 2/10 - Contrastive Loss: 3.1558


...

Epoch 9: 100%|██████████| 1026/1026 [03:49<00:00,  4.48it/s]


Epoch 9/10 - Contrastive Loss: 3.1120


Epoch 10: 100%|██████████| 1026/1026 [04:20<00:00,  3.94it/s]


Epoch 10/10 - Contrastive Loss: 3.1089


RuntimeError: Failed to run torchgraph see error message